In [16]:
import flappy_bird_env
import gymnasium as gym

In [17]:
env = gym.make("CartPole-v1")
# observation, info = env.reset()

# action = env.action_space.sample()
# observation, reward, terminated, truncated, info = env.step(action)
# if terminated or truncated:
#     observation, info = env.reset()
#     
# env.close()

In [18]:
import torch as th
import torch.nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

device = ("cuda" if th.cuda.is_available() else "mps" if th.backends.mps.is_available() else "cpu")

# Neural network for predicting action values
class CustomCNN(BaseFeaturesExtractor):

    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 128):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # CxHxW images (channels first)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        ).to(device)

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU()).to(device)
        
    def forward(self, observations: th.Tensor) -> th.Tensor:
        observations = observations.to(device)
        return self.linear(self.cnn(observations))


policy_kwargs = dict(
    features_extractor_class=CustomCNN,
)

# Initialize agent
model = PPO("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=0, device=device)

# Train agent
model.learn(total_timesteps=100)

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [1, 4]